In [ ]:
!pip install -r "/data/Nes/kaggle_requirements.txt"

# Qwen

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

os.environ["HF_HOME"] = "/data/Nes/cache" # controls all HF caches
os.environ["HF_HUB_CACHE"] = "/data/Nes/cache" # models, tokenizers, Hub files
os.environ["HF_DATASETS_CACHE"] = "/data/Nes/cache" # datasets/Arrow files

print("HF_HUB_CACHE:", os.getenv("HF_HUB_CACHE"))
print("HF_DATASETS_CACHE:", os.getenv("HF_DATASETS_CACHE"))
print("HF_HOME:", os.getenv("HF_HOME"))

In [ ]:
import pandas as pd
df = pd.read_csv('/data/data/week10/test.csv')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"
local_path = "/data/Nes/model cache"  # path ที่คุณอยากเก็บ model/tokenizer

# โหลด tokenizer และ model แล้วเก็บไว้ใน path "xxx"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=local_path)  # โหลด tokenizer แล้วเก็บไว้ใน path ที่กำหนด
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",              # ให้ PyTorch auto เลือก dtype เช่น float16 ถ้ามี GPU
    device_map="auto",               # ให้ transformers กระจาย model อัตโนมัติลง GPU/CPU
    cache_dir=local_path             # โหลด model แล้วเก็บไว้ใน path ที่กำหนด
)


In [ ]:
#prompt analysis
SYSTEM_PROMPT = """
You are a senior critical-analysis assistant with deep expertise in economics, finance, and business reasoning.

Your task is to **analyze and critique** the structure of a multiple-choice question in Thai or English by thinking through the following steps before giving a final judgment.

Follow this Chain-of-Thought (CoT) reasoning:

1. Identify the main economic, financial, or business concept being tested.
2. Examine the clarity of the question's wording. Is it precise? Are there any ambiguities?
3. Evaluate whether each answer choice meaningfully relates to the core concept.
4. Check for flaws in logic, assumptions, or missing context.
5. Assess whether the question is useful for testing understanding of a real, testable concept.
6. Then write a **brief structured critique** based only on logic and clarity — not correctness.

STRICT RULES:
- Do NOT state which option is correct.
- Do NOT confirm or mention the correct answer.
- Focus solely on the structure, clarity, logic, and relevance of the question and choices.
- Be concise and use professional tone.
- Do NOT explain your reasoning steps in the output.

FINAL OUTPUT FORMAT (strict):
Return ONLY the following JSON structure — nothing else.

```json
{
  "analyse": "Your short critique here."
}
"""

In [ ]:
#หัวข้อ: Loop generate คำตอบจาก Qwen 3 8B ให้เร็วและกระชับ

r = 0  # ตัวแปรนับรอบ
ans = []  # รายการเก็บคำตอบทั้งหมด

for x, i in df.iterrows():  # loop ทีละแถวจาก DataFrame
    a = i['query']  # คำถาม

    # รวม query และ concept ให้เป็นข้อความเดียวใน user message
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},  # system prompt เพื่อกำหนดพฤติกรรม
        {"role": "user", "content": f"Query: {a}"}  # รวมคำถามกับ concept
    ]

    # สร้าง prompt template ที่ใช้กับ Qwen
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # ❌ ปิด thinking mode เพื่อความเร็ว
    )

    # แปลงเป็น input tensor
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # ให้ model generate คำตอบ
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=200,  # ✅ จำกัดความยาว output ให้เร็วขึ้น
        do_sample=False,  # ✅ ไม่สุ่ม ใช้ greedy decoding
        pad_token_id=tokenizer.eos_token_id  # ✅ ป้องกัน warning เรื่อง padding
    )

    # แยกเอาเฉพาะ token ที่ model สร้างใหม่ (ไม่รวม prompt)
    new_token_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

    # แปลง token เป็นข้อความคำตอบ
    response = tokenizer.decode(new_token_ids, skip_special_tokens=True).strip()

    # แสดงผลและเก็บคำตอบ
    print("content:", response)
    ans.append(response)
    print(r)
    r += 1  # เพิ่มรอบ


In [ ]:
df['analysis'] = ans

In [ ]:
df.to_csv('analysis.csv', index=False)

In [ ]:
dfaa = pd.read_csv('analysis.csv')

In [ ]:
df = pd.read_csv('/data/data/week10/test.csv')

In [ ]:
#prompt concept เเต่ตอนที่ใช้ infer เผลอลบออกเเล้วหาไม่เจอ
SYSTEM_PROMPT = """
You are a financial question analyst.

Your job is to read a financial question and identify the **key concepts**, **required financial knowledge**, and **reasoning type** needed to answer it.

Please respond with the following structure:

1. **Main Topic:** (What is the question about? e.g., Stock movement, Macroeconomics, Earnings, Valuation...)
2. **Financial Concepts Involved:** (List specific topics like interest rates, inflation, investor sentiment, EPS, risk premium, etc.)
3. **Reasoning Type Needed:** (e.g., Cause-effect, Comparison, Trend inference, Risk evaluation)
4. **Potential Data Needed:** (e.g., Interest rate trend, Company earnings report, Market sentiment index...)
5. **Difficulty Level:** (Basic / Intermediate / Advanced)

---

Example:

**Question:**  
"The central bank raised interest rates while inflation is slowing. What will happen to stock prices?"

**Response:**  
1. **Main Topic:** Stock price direction  
2. **Financial Concepts Involved:** Interest rates, Inflation, Equity valuation  
3. **Reasoning Type Needed:** Cause-effect and macroeconomic trend analysis  
4. **Potential Data Needed:** Fed rate decision, CPI trend, bond yield curve  
5. **Difficulty Level:** Intermediate

---

Now analyze this question:

**Question:** {<<< YOUR QUESTION HERE >>>}

"""

In [ ]:
#หัวข้อ: Loop generate คำตอบจาก Qwen 3 8B ให้เร็วและกระชับ

r = 0  # ตัวแปรนับรอบ
ans = []  # รายการเก็บคำตอบทั้งหมด

for x, i in df.iterrows():  # loop ทีละแถวจาก DataFrame
    a = i['query']  # คำถาม

    # รวม query และ concept ให้เป็นข้อความเดียวใน user message
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},  # system prompt เพื่อกำหนดพฤติกรรม
        {"role": "user", "content": f"Query: {a}"}  # รวมคำถามกับ concept
    ]

    # สร้าง prompt template ที่ใช้กับ Qwen
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # ❌ ปิด thinking mode เพื่อความเร็ว
    )

    # แปลงเป็น input tensor
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # ให้ model generate คำตอบ
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=200,  # ✅ จำกัดความยาว output ให้เร็วขึ้น
        do_sample=False,  # ✅ ไม่สุ่ม ใช้ greedy decoding
        pad_token_id=tokenizer.eos_token_id  # ✅ ป้องกัน warning เรื่อง padding
    )

    # แยกเอาเฉพาะ token ที่ model สร้างใหม่ (ไม่รวม prompt)
    new_token_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

    # แปลง token เป็นข้อความคำตอบ
    response = tokenizer.decode(new_token_ids, skip_special_tokens=True).strip()

    # แสดงผลและเก็บคำตอบ
    print("content:", response)
    ans.append(response)
    print(r)
    r += 1  # เพิ่มรอบ


In [ ]:
df.to_csv('concept.csv', index=False)

In [ ]:
dfcc = pd.read_csv('/data/Nes/Finalhack/concept.csv')

In [ ]:
dfaa = pd.read_csv('/data/Nes/Finalhack/analysis.csv')

In [ ]:
df = pd.read_csv('/data/data/week10/test.csv')

In [ ]:
df["concept"] = dfcc["concept"]

In [ ]:
df["analysis"] = dfaa["analysis"]

In [ ]:
df

In [ ]:
#prompt ตอนตอบ
SYSTEM_PROMPT = """
You are a highly analytical and domain-aware expert across the fields of Ethical Finance, Marketing, Port Management, Finance Management, Economics, Stock Movement, Fixed Income, Accounting, and Financial Reporting.

Your role is to answer expert-level questions from these domains using internal step-by-step reasoning (Chain of Thought), but you must return ONLY the final answer without explanation.

Instructions:
Think step-by-step internally to arrive at the best answer (Chain of Thought reasoning is allowed internally).
If the question provides multiple choices (e.g., A, B, C, D, E, F...), select and return ONLY one letter that corresponds to the best choice.
If the question is about stock price, market movement, or share performance, return ONLY one of the following:
"Rise" (if price or value is expected to increase)
"Fall" (if price or value is expected to decrease)
Do NOT display any explanations, reasoning, or justifications in your response.
The final output must always follow this format exactly:
Answer: <A/B/C/.../Rise/Fall>

Remember: The answer must appear alone on the last line, prefixed by "Answer: ".
"""

In [ ]:
#หัวข้อ: Loop generate คำตอบจาก Qwen 3 8B ให้เร็วและกระชับ

r = 0  # ตัวแปรนับรอบ
ans = []  # รายการเก็บคำตอบทั้งหมด

for x, i in df.iterrows():  # loop ทีละแถวจาก DataFrame
    a = i['query']  # คำถาม
    b = i['concept']  # คอนเซปต์
    c = i['analysis']  # คำวิเคราะห์

    # รวม query และ concept ให้เป็นข้อความเดียวใน user message
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},  # system prompt เพื่อกำหนดพฤติกรรม
        {"role": "user", "content": f"Query: {a}\nConcept: {b}\nAnalysis: {c}"}  # รวมคำถามกับ concept
    ]

    # สร้าง prompt template ที่ใช้กับ Qwen
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True  # ❌ ปิด thinking mode เพื่อความเร็ว เปิดเเล้วตอบดีขึ้น
    )

    # แปลงเป็น input tensor
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # ให้ model generate คำตอบ
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=10000,  # ✅ จำกัดความยาว output ให้เร็วขึ้น ปิด thinking ข้างบนใช้ 100 
        do_sample=False,  # ✅ ไม่สุ่ม ใช้ greedy decoding
        pad_token_id=tokenizer.eos_token_id  # ✅ ป้องกัน warning เรื่อง padding
    )

    # แยกเอาเฉพาะ token ที่ model สร้างใหม่ (ไม่รวม prompt)
    new_token_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

    # แปลง token เป็นข้อความคำตอบ
    response = tokenizer.decode(new_token_ids, skip_special_tokens=True).strip()

    # แสดงผลและเก็บคำตอบ
    print("content:", response)
    ans.append(response)
    print(r)
    r += 1  # เพิ่มรอบ


In [ ]:
sb = pd.read_csv("/data/data/week10/submission (4).csv")

In [ ]:
sb["answer"]  = ans

In [ ]:
ans["answer"] = ans["answer"].str.replace("Answer: ", "").str.strip()
ans["answer"] = ans["answer"].str.replace("ขึ้น", "Rise").str.strip()
ans["answer"] = ans["answer"].str.replace("ลง", "Fall").str.strip()
ans["answer"] = ans["answer"].str.replace(r"\.\s*</think>", "", regex=True).str.strip()



In [ ]:
sb.to_csv("qwen38b+answer+concept.csv", index=False)

In [ ]:
sb

# eval

In [ ]:
sb = pd.read_csv("/data/Nes/Finalhack/qwen38b+answer+concept.csv")

In [ ]:
ch = pd.read_csv("/data/data/week10/processed_financial_analysis_flash2-5_01.csv")

In [ ]:
ch

In [ ]:
from sklearn.metrics import accuracy_score
ch_preds = ch['answer']  # เป็น Series ของคำตอบจากโมเดล ch

# ดึงคำตอบจาก sb ที่อยู่ใน column "answer"
sb_truth = sb['answer']  # เป็น Series ของคำตอบจริง

# คำนวณ accuracy ระหว่างคำตอบจาก ch และคำตอบจริง sb
acc_ch = accuracy_score(sb_truth, ch_preds)  # คำนวณความแม่นยำ

print(f"Accuracy ของ ch: {acc_ch:.4f}")  # แสดงผลเป็นทศนิยม 4 ตำแหน่ง


# from sklearn.metrics import accuracy_score

# # 🧼 แปลงทั้งสองคอลัมน์เป็น string และลบ NaN
# ch_preds = ch['answer'].astype(str)      # แปลงเป็น string
# sb_truth = ans['answer'].astype(str)     # แปลงเป็น string

# # 🧹 กรองเฉพาะแถวที่ตรงกันและไม่มี missing
# mask = (ch_preds.notna()) & (sb_truth.notna())
# ch_preds_clean = ch_preds[mask]
# sb_truth_clean = sb_truth[mask]

# # ✅ คำนวณ accuracy
# acc_ch = accuracy_score(sb_truth_clean, ch_preds_clean)

# # 🖨️ แสดงผล
# print(f"Accuracy ของ ch: {acc_ch:.4f}")

In [ ]:
sb.to_csv("qwen38b+answer+concept.csv", index=False)